## Creating snowflake connection and pushing data 

In [9]:
from sqlalchemy import create_engine
import pandas as pd


engine = create_engine(
    'snowflake://{user}:{password}@{account}/'.format(
        user='GEORGE9042',
        password='George9042',
        account='plninim-tg58176',
        warehouse='COMPUTE_WH',
        database='MY_TEST', 
        schema='NEW'
    )
)
sfOptions = {
    "sfURL": "https://lv50176.switzerland-north.azure.snowflakecomputing.com",
    "sfAccount": "plninim-tg58176",
    "sfUser": "GEORGE9042",
    "sfPassword": "George9042",
    "sfDatabase": "MY_TEST",
    "sfSchema": "NEW",
    "sfWarehouse": "COMPUTE_WH",
}
try:
    connection = engine.connect()
    connection.execute('USE ROLE ACCOUNTADMIN')
    connection.execute('USE DATABASE MY_TEST')
    connection.execute('USE SCHEMA NEW')

    #airports = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/airports.csv')
    #airports.to_sql('airports', con=connection, if_exists='replace',index = False,chunksize=16000)   

    #airport_frequencies = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/airport-frequencies.csv')
    #airport_frequencies.to_sql('airport_frequencies', con=connection, if_exists='replace',index = False,chunksize=16000) 

    #countries = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/countries.csv')
    #countries.to_sql('countries', con=connection, if_exists='replace',index = False,chunksize=16000)

    #airport_comments = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/airport-comments.csv')
    #airport_comments.to_sql('airport_comments', con=connection, if_exists='replace',index = False,chunksize=16000)

    #navaids = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/navaids.csv')
    #navaids.to_sql('navaids', con=connection, if_exists='replace',index = False,chunksize=16000)

    #regions = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/regions.csv')
    #regions.to_sql('regions', con=connection, if_exists='replace',index = False,chunksize=16000)

    #runways = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/runways.csv')
    #runways.to_sql('runways', con=connection, if_exists='replace',index = False,chunksize=16000)

    connection.execute('PUT file://../Airport_Pipeline/Geonames_data/all_countries.csv @~;')
    connection.execute('''COPY INTO geonames FROM @~ FILE_FORMAT = (TYPE = CSV, SKIP_HEADER = 1) ON_ERROR=CONTINUE;''')
    

finally:
    connection.close()
    engine.dispose()
    
    

### 1. How many airports, airfields and heliports exist in each country and continent?

In [20]:
from sqlalchemy import create_engine
import pandas as pd


engine = create_engine(
    'snowflake://{user}:{password}@{account}/'.format(
        user='GEORGE9042',
        password='George9042',
        account='plninim-tg58176',
        warehouse='COMPUTE_WH',
        database='MY_TEST', 
        schema='NEW'
    )
)
try:
    connection = engine.connect()
    connection.execute('USE ROLE ACCOUNTADMIN')
    connection.execute('USE DATABASE MY_TEST')
    connection.execute('USE SCHEMA NEW')
    clean_continent = '''UPDATE airports SET CONTINENT = COALESCE(CONTINENT, 'NA')'''
    clean_country = '''UPDATE airports SET iso_country = COALESCE(iso_country, 'NA')'''

    query_continent = '''select * from airport_per_continent'''
    query_continent = pd.read_sql(query_continent,connection)

    query_avg_air_ele = '''select * from AVG_AIRPORT_ELEVATION_COUNTRY'''
    query_avg_air_ele = pd.read_sql(query_avg_air_ele,connection)

    min_max_ele = '''select * from MAX_MIN_ELEVATION''' 
    min_max_ele= pd.read_sql(min_max_ele,connection)
    
    query_country='''select * from airport_per_country '''
    query_country = pd.read_sql(query_country,connection)

    total_population_country = '''select * from total_population_country'''
    total_population_country = pd.read_sql(total_population_country,connection)

    city_towns_with_airport = '''select * from city_towns_with_airport'''
    city_towns_with_airport = pd.read_sql(city_towns_with_airport,connection)

    city_towns_with_geoname = '''select * from city_towns_with_geoname'''
    city_towns_with_geoname = pd.read_sql(city_towns_with_geoname,connection)

    min_max_avg_city_elevation = '''select * from min_max_avg_city_elevation'''
    min_max_avg_city_elevation = pd.read_sql(min_max_avg_city_elevation,connection)

    lowest_elevation_city = '''select * from lowest_elevated_city'''
    lowest_elevation_city = pd.read_sql(lowest_elevation_city,connection)

    highest_elevation_city = '''select * from highest_elevated_city'''
    highest_elevation_city = pd.read_sql(highest_elevation_city,connection)

    print(highest_elevation_city)
    
finally:
    connection.close()
    engine.dispose()

           name  elevation
0  GreenSite HQ     3200.0


## Creating Snowflake Tables

In [ ]:
create or replace TABLE MY_TEST.NEW.AIRPORT_COMMENTS (
	ID NUMBER(38,0),
	threadRef FLOAT,
	airportRef NUMBER(38,0),
	airportIdent VARCHAR(16777216),
	date VARCHAR(16777216),
	memberNickname VARCHAR(16777216),
	subject VARCHAR(16777216),
	body VARCHAR(16777216),
    PRIMARY KEY (ID)
);

create or replace TABLE MY_TEST.NEW.AIRPORTS (
	ID NUMBER(38,0) NOT NULL,
	IDENT VARCHAR(16777216),
	TYPE VARCHAR(16777216),
	NAME VARCHAR(16777216),
	LATITUDE_DEG FLOAT,
	LONGITUDE_DEG FLOAT,
	ELEVATION_FT NUMBER(38,0),
	CONTINENT VARCHAR(16777216),
	ISO_COUNTRY VARCHAR(16777216),
	ISO_REGION VARCHAR(16777216),
	MUNICIPALITY VARCHAR(16777216),
	SCHEDULED_SERVICE VARCHAR(16777216),
	GPS_CODE VARCHAR(16777216),
	IATA_CODE VARCHAR(16777216),
	LOCAL_CODE VARCHAR(16777216),
	HOME_LINK VARCHAR(16777216),
	WIKIPEDIA_LINK VARCHAR(16777216),
	KEYWORDS VARCHAR(16777216),
	primary key (ID),
	foreign key (ISO_REGION) references MY_TEST.NEW.REGIONS(CODE)
);

create or replace TABLE MY_TEST.NEW.RUNWAYS (
	ID NUMBER(38,0) NOT NULL,
	AIRPORT_REF NUMBER(38,0),
	AIRPORT_IDENT VARCHAR(16777216),
	LENGTH_FT FLOAT,
	WIDTH_FT FLOAT,
	SURFACE VARCHAR(16777216),
	LIGHTED BOOLEAN,
	CLOSED BOOLEAN,
	LE_IDENT VARCHAR(16777216),
	LE_LATITUDE_DEG FLOAT,
	LE_LONGITUDE_DEG FLOAT,
	LE_ELEVATION_FT NUMBER(38,0),
	LE_HEADING_DEGT FLOAT,
	LE_DISPLACED_THRESHOLD_FT NUMBER(38,0),
	HE_IDENT VARCHAR(16777216),
	HE_LATITUDE_DEG FLOAT,
	HE_LONGITUDE_DEG FLOAT,
	HE_ELEVATION_FT NUMBER(38,0),
	HE_HEADING_DEGT FLOAT,
	HE_DISPLACED_THRESHOLD_FT NUMBER(38,0),
	primary key (ID),
	foreign key (AIRPORT_REF) references MY_TEST.NEW.AIRPORTS(ID)
);

create or replace TABLE MY_TEST.NEW.REGIONS (
	CODE VARCHAR(16777216) NOT NULL,
	LOCAL_CODE VARCHAR(16777216),
	ISO_COUNTRY VARCHAR(16777216),
	NAME VARCHAR(16777216),
	CONTINENT VARCHAR(16777216),
	primary key (CODE)
);

create or replace TABLE MY_TEST.NEW.NAVAIDS (
	ID_NAV NUMBER(38,0) NOT NULL,
	FILENAME VARCHAR(16777216),
	IDENT VARCHAR(16777216),
	NAME VARCHAR(16777216),
	TYPE VARCHAR(16777216),
	FREQUENCY_KHZ FLOAT,
	LATITUDE_DEG FLOAT,
	LONGITUDE_DEG FLOAT,
	ELEVATION_FT NUMBER(38,0),
	ISO_COUNTRY VARCHAR(16777216),
	DME_FREQUENCY_KHZ FLOAT,
	DME_CHANNEL NUMBER(38,0),
	DME_LATITUDE_DEG FLOAT,
	DME_LONGITUDE_DEG FLOAT,
	DME_ELEVATION_FT NUMBER(38,0),
	SLAVED_VARIATION_DEG FLOAT,
	MAGNETIC_VARIATION_DEG FLOAT,
	USAGE_TYPE VARCHAR(16777216),
	POWER FLOAT,
	ASSOCIATED_AIRPORT VARCHAR(16777216),
	ASSOCIATED_AIRPORT_IDENT VARCHAR(16777216),
	primary key (ID_NAV)
);

create or replace TABLE MY_TEST.NEW.COUNTRIES (
	ID NUMBER(38,0),
	CODE VARCHAR(16777216),
	NAME VARCHAR(16777216),
	CONTINENT VARCHAR(16777216),
	WIKIPEDIA_LINK VARCHAR(16777216),
	KEYWORDS VARCHAR(16777216)
	primary key (ID)
);

create or replace TABLE MY_TEST.NEW.AIRPORT_COMMENTS (
	ID NUMBER(38,0),
	threadRef FLOAT,
	airportRef NUMBER(38,0),
	airportIdent VARCHAR(16777216),
	DATE VARCHAR(16777216),
	memberNickname VARCHAR(16777216),
	SUBJECT VARCHAR(16777216),
	BODY VARCHAR(16777216)\
	PRIMARY KEY (ID)
);


### Views used in snowflake


In [ ]:
airport_per_continent = '''create or replace view AIRPORT_PER_CONTINENT as
    -- comment = '<comment>'
    SELECT continent,    
       COUNT(CASE WHEN type IN ('seaplane_base','medium_airport', 'large_airport') THEN 1 END) AS airports,
       COUNT(CASE WHEN type = 'heliport' THEN 1 END) AS heliport,
       COUNT(CASE WHEN type = 'small_airport' THEN 1 END) AS airfield       
       FROM airports
       GROUP BY continent;'''

airport_per_country = '''create or replace view AIRPORT_PER_COUNTRY as
SELECT iso_country,
/* assumumes the small airport is an airfield*/
       COUNT(CASE WHEN type IN ('seaplane_base','medium_airport', 'large_airport') THEN 1 END) AS airports,
       COUNT(CASE WHEN type = 'heliport' THEN 1 END) AS heliport,
       COUNT(CASE WHEN type = 'small_airport' THEN 1 END) AS airfield       
       FROM airports
       GROUP BY iso_country;'''


min_max_elevation = '''create or replace view MY_TEST.NEW.MAX_MIN_ELEVATION(
	TYPE,
	HIGHEST_ELEVATION,
	LOWEST_ELEVATION
) as
SELECT type,
       MAX(elevation_ft) AS highest_elevation,
       MIN(elevation_ft) AS lowest_elevation
FROM airports
GROUP BY type;'''

avg_airport_elevation = '''create or replace view MY_TEST.NEW.AVG_AIRPORT_ELEVATION_COUNTRY(
	ISO_COUNTRY,
	AVG_AIRPORT_ELEVATION,
	AVG_AIRFIELD_ELEVATION,
	AVG_HELIPORT_ELEVATION
) as
SELECT iso_country,
       AVG(CASE WHEN type IN ('seaplane_base','medium_airport', 'large_airport') THEN elevation_ft END) AS avg_airport_elevation,
       AVG(CASE WHEN type = 'small_airport' THEN elevation_ft END) AS avg_airfield_elevation,
       AVG(CASE WHEN type = 'heliport' THEN elevation_ft END) AS avg_heliport_elevation
FROM airports
GROUP BY iso_country;'''

total_population_country = '''create or replace view MY_TEST.NEW.TOTAL_POPULATION_COUNTRY(
	TOTAL_POPULATION,
	COUNTRY_CODE
) as 
select sum(population) as total_population, country_code from geonames group by country_code;'''

city_towns_with_airport = '''create or replace view MY_TEST.NEW.CITY_TOWNS_WITH_AIRPORT(
	CITY_TOWNS_USING_AIRPORT,
	ISO_COUNTRY
) as
select count(distinct(municipality)) as city_towns_using_airport, iso_country from airports group by iso_country order by city_towns_using_airport desc;'''

city_towns_with_geoname = '''create or replace view MY_TEST.NEW.CITY_TOWNS_WITH_GEONAME(
	CITY_TOWNS_USING_GEONAME,
	COUNTRY_CODE
) as
select count(distinct(name)) as city_towns_using_geoname, country_code from geonames group by country_code order by city_towns_using_geoname desc;'''


min_max_avg_city_elevation = '''create or replace view MY_TEST.NEW.MIN_MAX_AVG_CITY_ELEVATION(
	MINIMUM_ELEVATION,
	MAXIMUM_ELEVATION,
	AVERAGE_ELEVATION,
	COUNTRY_CODE
) as 
select min(elevation) as minimum_elevation,
max(elevation) as maximum_elevation,
ROUND(avg(elevation), 2)as average_elevation,
country_code
from geonames
WHERE elevation IS NOT NULL
group by country_code 
order by country_code;'''